In [46]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [47]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df.sample(20)

,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
240340,0xe8ceba8fa2f8cdc250130c7abadc4629eac880aa,in,0x574d4ac4448392c9600c5c19e26a1ddb55cfbc19,NaN,transfer,3.028514e+04,1,1,Ethereum
159985,0xacdd6e384288a6f2c5245f8ac6a6200f05956a30,out,0x96af8a5cae82376e966479c740bb38732c30a662,NaN,transfer,1.140963e+04,2,2,Ethereum
172313,0x24a73ecbce88064b34807dcf9b157aac663c973d,out,0x6dc4cfbbf3cea7870b7900b38d708b4207960f12,NaN,0x,1.867144e+04,2,2,Ethereum
217341,0xffec0067f5a79cff07527f63d83dd5462ccf8ba4,out,0x520b48dfcbd7f37fb391a1e33b1b0517c8f03869,NaN,transfer,1.519380e+04,1,1,Ethereum
88424,0x8809b98bdade1a96b508b891cf9e008e2d94ff7b,out,0x862ccbe390023a0243b013ac8ca8dfa706197dfd,NaN,transfer,1.384065e+04,19,19,Ethereum
73500,0xb0b0f6f13a5158eb67724282f586a552e75b5728,out,0xd050e0a4838d74769228b49dff97241b4ef3805d,NaN,yoink,6.191116e+04,1,1,Ethereum
222974,0xc17d043df0399347f206a397b13500fc22035a40,in,0x12c3fe56d728db837e2d910f8f695689265f4045,NaN,transfer,5.035410e+04,1,1,Ethereum
252986,0xc2a4998c7d4c4f5c1d31c6c84a3887f73b9a3306,out,0x06d3a30cbb00660b85a30988d197b1c282c6dcb6,NaN,transfer,2.014164e+06,1,1,Ethereum
193161,0x88d8da5867688214586d86a5b97c37e45b1e74d2,out,0x8c4e375fa84705874851d220bb9511b3261bf70f,NaN,transfer,1.069417e+05,2,2,Ethereum
341,0x6f15ee9258acdebf356db7ab607bb255a00c6fdf,in,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3,NaN,0x76149b98,8.424363e+04,10,9,Arbitrum


In [48]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [49]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby('address')[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1126789040.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [50]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action',\
                'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share','address']

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [51]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [52]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'address':'nunique'
})
df_action.name = 'df_action'

# display(df_action)

In [53]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

# df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

In [54]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

Dollar-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/345832494.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
672,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,0.908970,0.136786
671,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,0.544242,0.071865
37,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,0.525326,0.000000
673,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,0.471929,0.082285
125,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,0.381631,0.000000
678,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,0.272238,0.050338
130,Arbitrum,gmx,gmx - Vault | 0x364e2311,0.217463,0.000000
1,Arbitrum,OTHER,OTHER - OTHER | OTHER,0.216843,0.034668
136,Arbitrum,gmx,gmx - Vault | swap,0.142702,0.000106
8,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,0.133860,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/345832494.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/345832494.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
23,Optimism,Celer,Celer - Bridge | addLiquidity,1.002681,0.000000
263,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,0.883819,0.134040
269,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,0.200170,0.031566
69,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,0.156633,0.041367
26,Optimism,Celer,Celer - Bridge | send,0.103441,0.000000
74,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,0.102830,0.000000
96,Optimism,OTHER,OTHER - OTHER | OTHER,0.090966,0.018323
104,Optimism,Perpetual Protocol,Perpetual Protocol - Vault | depositFor,0.074724,0.000000
290,Optimism,Velodrome,Velodrome - Pair | addLiquidity,0.072420,0.000000
4,Optimism,Aave,Aave - AToken | repay,0.069251,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/345832494.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
38,Optimism,Uniswap,1.314341,0.215869
7,Optimism,Celer,1.148165,0.205179
17,Optimism,Kyberswap,0.292208,0.044174
1,Optimism,Aave,0.146573,0.035010
39,Optimism,Uniswap V3,0.139361,0.028245
34,Optimism,Sonne Finance,0.137208,0.023588
40,Optimism,Velodrome,0.094357,0.012420
22,Optimism,OTHER,0.090966,0.018323
26,Optimism,Perpetual Protocol,0.087592,0.013292
9,Optimism,Curve,0.078605,0.008993


In [55]:
print('Address-Weighted')
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        tdf = tdf[tdf['address'] >=20]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        # print('Sum by App')
        # sumdf = tdf.groupby(['chain','project']).sum()
        # sumdf.reset_index(inplace=True)
        # sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        # sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        # sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        # sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        # display( sumdf.head(10) )

Address-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/903782907.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
7,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,51.880684,0.000000
15,Arbitrum,sushi,sushi - Pair | swapExactTokensForETH,43.063953,0.000000
13,Arbitrum,gmx,gmx - sGMX | 0xf3daeacc,35.380614,0.000000
3,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,33.906252,0.000000
9,Arbitrum,gmx,gmx - Vault | 0x364e2311,31.299871,0.000000
18,Arbitrum,uniswap_v3,uniswap_v3 - Pair | mint,25.269913,0.000000
19,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,23.965963,20.857980
5,Arbitrum,celer_network,celer_network - Bridge | send,17.962489,0.000000
11,Arbitrum,gmx,gmx - Vault | swap,15.316558,0.075303
20,Arbitrum,uniswap_v3,uniswap_v3 - Pair | uniswapV3Swap,13.697262,9.424339


Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/903782907.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
128,Ethereum,matic,matic - StakeManager | buyVoucher,89.481270,1.496797
105,Ethereum,gnosis_safe,gnosis_safe - GnosisSafev1.3.0 | transfer,71.753041,0.000000
80,Ethereum,curvefi,curvefi - threepool_swap | exchange,68.524750,59.496168
0,Ethereum,OTHER,OTHER - OTHER | OTHER,66.735365,71.071145
118,Ethereum,lido,lido - steth | submit,62.102421,0.000000
48,Ethereum,chainlink,chainlink - Staking | transferAndCall,61.636517,0.000000
177,Ethereum,tokenlon_v2,tokenlon_v2 - AMMWrapperWithPath | toAMM,60.525237,75.458469
22,Ethereum,apecoin,apecoin - ApeCoinStaking | depositSelfApeCoin,60.356324,0.000000
109,Ethereum,gnosis_safe,gnosis_safe - Safev1.1.1 | transfer,59.656901,0.000000
72,Ethereum,curvefi,curvefi - frax_base_pool_fraxbp | add_liquidity,52.662123,0.000000


Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/903782907.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
10,Optimism,Velodrome,Velodrome - Pair | addLiquidity,22.228758,0.000000
2,Optimism,Aave,Aave - AToken | repay,20.102279,0.000000
6,Optimism,Celer,Celer - Bridge | send,19.440471,0.000000
7,Optimism,OTHER,OTHER - OTHER | OTHER,11.095232,11.686569
3,Optimism,Aave,Aave - AToken | supply,10.575471,0.000000
9,Optimism,Uniswap,Uniswap - Pool | multicall,8.024692,6.789021
0,Optimism,1inch,1inch - AggregationRouterV5 | swap,2.481089,16.355563
1,Optimism,Aave,Aave - AToken | borrow,0.000000,13.263009
4,Optimism,Aave,Aave - AToken | withdraw,0.000000,27.050186
5,Optimism,Celer,Celer - Bridge | relay,0.000000,14.671806
